In [ ]:
import os
from contextlib import redirect_stdout

import sys
sys.path.append('./coeqwalpackage')

import numpy as np
import pandas as pd
import datetime as dt
import re
from coeqwalpackage.metrics import *
from coeqwalpackage.tier import generate_tier_assignment_matrix
import cqwlutils as cu
import plotting as pu

## Init

In [ ]:
CtrlFile = 'CalSim3DataExtractionInitFile_v4.xlsx'
CtrlTab = 'Init'

ScenarioListFile, ScenarioListTab, ScenarioListPath, DVDssNamesOutPath, SVDssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, DVDssMin, DVDssMax, SVDssMin, SVDssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax = cu.read_init_file(CtrlFile, CtrlTab)

df, dss_names = read_in_df(ConvertDataOutPath,DVDssNamesOutPath)

df.columns = ['_'.join(map(str, col)) if isinstance(col, tuple) else col for col in df.columns]


## Define variables

In [ ]:
hist_percentiles=[0.2, 0.333, 0.5] # percentiles for historical record
tier_thresholds=(0.9, 0.67, 0.3) # tier assignment thresholds
cdec_calsim_vars = {
    "SHA": "S_SHSTA",
    "CLE": "S_TRNTY",
    "ORO": "S_OROVL",
    "FOL": "S_FOLSM",
    "NML": "S_MELON",
    "MIL": "S_MLRTN",
    "LUS": "S_SLUIS_SWP",
    "SLF": "S_SLUIS_CVP"
} # mapping between cdec and calsim var names

cdec_file = "CDEC_Station_Key.csv" # cdec key file name
metadata_file = "reservoir_metadata_table.csv" # reservoir metadata output file name

## Aux functions

In [ ]:
# find_calsim_model_root() is now in cqwlutils (use cu.find_calsim_model_root())
# generate_tier_assignment_matrix() is now imported from coeqwalpackage.tier

def get_relative_folder(full_folder_path, known_tail):
    """Convert absolute path to relative path based on known tail."""
    full_folder_path = os.path.normpath(full_folder_path)
    known_tail = os.path.normpath(known_tail)
    
    # Find where the known folder structure starts
    idx = full_folder_path.lower().find(known_tail.lower())
    
    if idx == -1:
        raise ValueError("Known tail not found in full path.")
    
    relative_suffix = full_folder_path[idx:]
    
    return os.path.join("..", "..", relative_suffix)

In [ ]:
base_model_dir = cu.find_calsim_model_root()

hist_full_subdir = os.path.join(base_model_dir, "Scenarios", "CDEC_Historical_Monthly_Storage")
known_tail = r"CalSim3_Model_Runs\Scenarios\CDEC_Historical_Monthly_Storage"

hist_subdir = get_relative_folder(hist_full_subdir, known_tail)
print("hist_subdir: " + hist_subdir)

key_file = os.path.join(hist_subdir, cdec_file)
cdec_df = pd.read_csv(key_file)
cdec_df.columns = cdec_df.columns.str.strip()

cdec_df["CalSim_Variable"] = cdec_df["CDEC_Key"].map(cdec_calsim_vars)
cdec_df = cdec_df.dropna(subset=["CalSim_Variable"])

file_prefix = "20250506_MonthlyResData_CDEC-"
cdec_df["filename"] = file_prefix + cdec_df["CDEC_Key"] + ".csv"
cdec_df["file_path"] = cdec_df["filename"].apply(lambda f: os.path.join(hist_subdir, f))

final_table = cdec_df[["ReservoirName", "CDEC_Key", "CalSim_Variable", "filename", "file_path"]]
print(final_table)

# check if output directory exists
if not os.path.exists(hist_subdir):
    # print warning
    print("Warning: directory " + hist_subdir + " does not exists and will be created")
    
    # Create the directory
    os.makedirs(hist_subdir)
    
output_path = os.path.join(hist_subdir, metadata_file)
final_table.to_csv(output_path, index=False)
print(f"\n Metadata table saved to: {output_path}")

In [ ]:
tier_df = generate_tier_assignment_matrix(
    # df, cdec_df, hist_data_dir,
    df, cdec_df, hist_subdir,
    percentiles=hist_percentiles,
    tier_thresholds=tier_thresholds
)


In [ ]:
tier_df

In [ ]:
print("Done!")